In [3]:
from sentence_transformers import SentenceTransformer
import torch

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("gpu num: ", n_gpu)

gpu num:  1


In [5]:
embedder = SentenceTransformer('bert-base-nli-stsb-mean-tokens')

In [16]:
embedder.cuda()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [17]:
import json
from tqdm.notebook import tqdm
import pickle

In [27]:
import os

def read_json(file_path):
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)
    return data

def write_file(file_name, embeddings):
    sent_embedding_folder = './sentence_embeddings/'
    if not os.path.exists(sent_embedding_folder):
        os.mkdir(sent_embedding_folder)
        
    dest_path = os.path.join(
        sent_embedding_folder, file_name +  '_new_dialogsum_sentence_transformer.pkl'
    )
    with open(dest_path, 'wb') as f:
        pickle.dump(embeddings, f)

def encode_conversation(file_path):
    file_name = file_path.split('/')[-1].split('.')[0]
    print(f'Encoding {file_name} split.')
    data = read_json(file_path)
    sent = []
    for i in range(0, len(data)):
        if len(data[i]['dialogue'].split('\r\n')) > 1:
            sentences = data[i]['dialogue'].split('\r\n')
        else:
            sentences = data[i]['dialogue'].split('\n')
        sent.append(sentences)
    
    embeddings = []
    with torch.no_grad():
        for i in tqdm(range(0, len(sent))):
            embedding = embedder.encode(sent[i])
            embeddings.append(embedding)
    
    write_file(file_name, embeddings)

In [31]:
data_folder = './new_dialogsum/'
train_path = os.path.join(data_folder, 'train.json')
val_path = os.path.join(data_folder, 'val.json')
test_path = os.path.join(data_folder, 'test.json')
paths = [train_path, dev_path, test_path]

In [32]:
for path in paths:
    encode_conversation(path)

Encoding val split.


/home/tnguyen/dialogue-text-summarization-dokument/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/495 [00:00<?, ?it/s]

Encoding test split.


  0%|          | 0/496 [00:00<?, ?it/s]